# BC 3
## Single tiny network; 
## Input = sub-regions 

In [38]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from scipy import stats
from sklearn.metrics import accuracy_score

In [39]:
NUM_TN = 5
SUB_REGION_SCALE = 14

#dataset specific parameters
NUM_CLASS = 10

In [40]:
#load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [41]:
#Generate cropped train image
X_train_cropped = []
y_train_cropped = []
for i in range (X_train.shape[0]):
  image_cropped = tf.image.random_crop(X_train[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE])
  image_cropped = np.array(image_cropped)
  X_train_cropped.append(image_cropped)
  y_train_cropped.append(y_train[i])
X_train_cropped = np.array(X_train_cropped)
y_train_cropped = np.array(y_train_cropped)

In [42]:
#Generate cropped test image
X_test_cropped_list = []
y_test_cropped_list = []

for i in range(NUM_TN):
  X_test_cropped = []
  y_test_cropped = []
  for i in range (X_test.shape[0]):
    image_cropped = tf.image.random_crop(X_test[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE])
    image_cropped = np.array(image_cropped)
    X_test_cropped.append(image_cropped)
    y_test_cropped.append(y_test[i])
  X_test_cropped = np.array(X_test_cropped)
  y_test_cropped = np.array(y_test_cropped)

  X_test_cropped_list.append(X_test_cropped)
  y_test_cropped_list.append(y_test_cropped)

In [43]:
#normalizing dataset
X_train_cropped = X_train_cropped/255

for i in range(NUM_TN):
  X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [44]:
#transform to vector
X_train_cropped = X_train_cropped.reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE))

for i in range(NUM_TN):
  X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE))

In [45]:
#one-hot encoding
y_train_cropped = to_categorical(y_train_cropped)

for i in range(NUM_TN):
  y_test_cropped_list[i] = to_categorical(y_test_cropped_list[i])

In [46]:
#build ANN model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE))
model.add(Dense(64, activation='relu'))
model.add(Dense(NUM_CLASS, activation = 'softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
#train model
hist = model.fit(X_train_cropped, y_train_cropped, validation_data=(X_test_cropped_list[0], y_test_cropped_list[0]), epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 2.0717 - accuracy: 0.2515 - val_loss: 1.5040 - val_accuracy: 0.4871
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 1.4182 - accuracy: 0.5181 - val_loss: 1.2912 - val_accuracy: 0.5554
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 1.2458 - accuracy: 0.5778 - val_loss: 1.2011 - val_accuracy: 0.5911
Epoch 4/5
469/469 [==============================] - 1s 3ms/step - loss: 1.1681 - accuracy: 0.6060 - val_loss: 1.1552 - val_accuracy: 0.6081
Epoch 5/5
469/469 [==============================] - 1s 3ms/step - loss: 1.1035 - accuracy: 0.6259 - val_loss: 1.0905 - val_accuracy: 0.6270


## Model Evaluation

In [48]:
# Predict on test image
y_pred_list = []
for i in range(NUM_TN):
  y_pred = model.predict(X_test_cropped_list[i])
  y_pred_list.append(y_pred)
    
# Ensemble voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.argmax(np.sum(y_pred_list, axis=0), axis=1)

In [49]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.8282